# Public dataset evaluation

In this notebook we test and evaluabte publicly available datasets (see *datasets/* folder).

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib notebook
%reload_ext autoreload
%autoreload 2

from public_data_utils import *

plt.rcParams['figure.figsize'] = 7, 3
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# set the random seed so can reproduce when something didn't work. 
# (only when cells are run in order)
np.random.seed(1)

This notebook contains functionality to evaluate the proposed algorithms on real datasets. To simply reproduce the plots from the paper you can directly run GenerateAllFigures. 

# Preparation

### 1. Download .mat files and save them in folder ./datasets/ (can simply run download_datasets.sh)

WiFi: http://www.robesafe.es/repository/UAHWiFiDataset/

Lawnmower: https://panda.frc.ri.cmu.edu/projects/emergencyresponse/RangeData/download.html

See datasets/README.md for file description.  

### 2. Choose dataset and range

In [ ]:
#filename = 'datasets/uah1.mat' # fingers
filename = 'datasets/Plaza1.mat'; # zig zag. 
#filename = 'datasets/Plaza2.mat' # triangle
#filename = 'datasets/Gesling1.mat' # not working
#filename = 'datasets/Gesling2.mat' # not working
#filename = 'datasets/Gesling3.mat' # 

full_df, anchors_df, traj = read_dataset(filename)
xlim, ylim = get_plotting_params(filename)
print(xlim, ylim)

### 3. Prepare dataset

In [ ]:
%matplotlib notebook
#%matplotlib inline
fig, axs = plt.subplots(1, 2)
plot_df = full_df
sns.scatterplot(data=plot_df, x='px', y='py', hue='timestamp', linewidth=0.0, ax=axs[0])
sns.scatterplot(data=plot_df, x='timestamp', y='py', hue='timestamp', linewidth=0.0, ax=axs[1])
sns.scatterplot(data=anchors_df, x='px', y='py', linewidth=0.0,  ax=axs[0], color='red')

### 4. (optional) plot distance measurements

In [ ]:
fig, axs = plt.subplots(1, 2)
range_df = full_df.loc[full_df.system_id == range_system_id].copy()
sns.scatterplot(data=range_df, x='px', y='py', hue='timestamp', linewidth=0.0, ax=axs[0])
sns.scatterplot(data=anchors_df, x='px', y='py', linewidth=0.0,  ax=axs[0], color='red')
for a_id, px, py in zip(anchors_df.anchor_id, anchors_df.px, anchors_df.py):
    print(a_id, px, py)
    axs[0].annotate(s='a{}'.format(a_id), xy=(px+2,py+2), color='red')
axs[0].legend('')
sns.scatterplot(data=range_df, x='timestamp', y='px', hue='timestamp', linewidth=0.0, ax=axs[1])

plot_distance_times(full_df)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(5, 2)
ax = plot_distance_errors(full_df, ax=ax)
#savefig(fig, 'results/accuracy.pdf')

In [ ]:
anchor_names = sorted(range_df.anchor_name.unique())
print(anchor_names)
fig, axs = plt.subplots(1, len(anchor_names), sharey=True)
fig.set_size_inches(15, 4)
for ax, anchor_name in zip(axs, anchor_names):
    plot_df = range_df.loc[range_df.anchor_name==anchor_name].copy()
    plot_df.loc[:, 'distance error'] = plot_df.distance.values - plot_df.distance_gt.values
    plot_df.loc[:, 'anchor name'] = plot_df.anchor_name.values
    anchors_df.loc[:, 'anchor name'] = anchors_df.anchor_name.values
    sns.scatterplot(data=plot_df, x='px', y='py', hue='anchor name', size='distance error',
                    hue_order=anchor_names, linewidth=0.0, alpha=0.8, ax=ax, legend=False)
    anchors_df = anchors_df.apply(pd.to_numeric, downcast='float', errors='ignore', axis=0)
    sns.scatterplot(data=anchors_df, x='px', y='py', hue='anchor name',
                    linewidth=0.0, legend=False, ax=ax)
    ax.axis('equal')
    ax.set_title(anchor_name)
g = sns.scatterplot(data=anchors_df, x='px', y='py', hue='anchor name',
                    linewidth=0.0, legend='full', ax=ax)
g.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
fig.suptitle('Scatter plots with size proportional to distance error.')

# Global Algorithms

### Plot and print results from generate_results.py

In [ ]:
# convert all numerical columns to float, ignore non-numeric.
fname = 'results/algorithms_sunday.pkl'
result_df = pd.read_pickle(fname)
result_df = result_df.apply(pd.to_numeric, errors='ignore')
#print_table = result_df[result_df.n_measurements.isin([40, 100, 200, 300, 499])]
print_table = result_df[(result_df.n_complexity >= 5) & (result_df.n_measurements >= 100)]
pt = pd.pivot_table(print_table, values='mse', index='method', columns=['n_measurements', 'n_complexity'], 
               aggfunc=['mean', 'std']).reindex(['gt','srls raw', 'srls', 'rls raw', 'rls', 'lm-ellipse', 'lm-ours-weighted',
                                                 'ours', 'ours-weighted'])

def highlight_min(data, exclude=[0], color='red', index=0):
    """
    :param exclude: rows indices to exclude for calculating min.
    :param index: set to 0 for smallest, 1 for second smallest, etc.
    """
    attr = 'background-color: {}'.format(color)
    
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        indices = [i for i in range(len(data)) if i not in exclude]
        is_min = data == sorted(data[indices])[index]
        if sum(is_min) > 1:
            return ['']*len(data)
        return [attr if v else '' for v in is_min]
    
def highlight_both(data, exclude=[0]):
    attr1 = highlight_min(data, exclude=exclude, color='red', index=0)
    attr2 = highlight_min(data, exclude=exclude, color='orange', index=1)
    return [a1+a2 for a1, a2 in zip(attr1, attr2)]
    
styler = pt.style.apply(highlight_min, axis=0)
pt = pt.style.apply(highlight_both, axis=0)
pd.set_option('precision', 2)
pd.set_option('max_columns', 100)
pt

In [ ]:
plot_df = result_df[result_df.mae < 100]
plot_df = plot_df[plot_df.n_measurements > 100]
fg = sns.FacetGrid(data=plot_df, col='n_complexity', hue='method', legend_out=True)
fg.map(plt.semilogy, 'n_measurements', 'mae', linestyle='', marker='.', alpha=0.5)
legend = plt.gca().get_legend()
plt.legend()

In [ ]:
# understand why N=100, K=19 is so bad...
df = result_df.loc[(result_df.n_measurements==100) & (result_df.n_complexity==19), :]
fig, ax = plt.subplots()
for method, df_m in df.groupby('method'):
    ax.scatter(df_m.n_it, df_m.mse, label=method)
ax.set_yscale('log')
ax.legend(loc='upper right')

# Sandbox

Space to try out stuff

In [ ]:
from evaluate_dataset import compute_distance_matrix, compute_anchors
chosen_distance = 'distance'
#chosen_distance = 'distance_gt'
anchor_names = None

## Construct anchors. 
anchors = compute_anchors(anchors_df, anchor_names)
print(anchors.shape)

## Construct times.
times = full_df[full_df.system_id == range_system_id].timestamp.unique()

## Construct D.
D, times = compute_distance_matrix(full_df, anchors_df, anchor_names, times, chosen_distance)
print(D.shape)
if np.sum(D > 0) > D.shape[0]:
    print('Warning: multiple measurements for times:{}/{}!'.format(
          np.sum(np.sum(D > 0, axis=1)>1), D.shape[0]))

## Construct ground truth.
points_gt = get_ground_truth(full_df, times)

In [ ]:
from other_algorithms import apply_algorithm
from fit_curve import fit_trajectory
print(D.shape)
fig, ax = plt.subplots()
ax.scatter(points_gt.px, points_gt.py, s=10)

traj.set_n_complexity(2)

coeffs, __, __ = apply_algorithm(traj, D, times, anchors, method='ours-weighted')
traj.set_coeffs(coeffs=coeffs)
traj.plot_pretty(times=times, ax=ax, color='red', label='fitted')

traj.print()
coeffs = fit_trajectory(points_gt.T, times, traj)
traj.set_coeffs(coeffs=coeffs)
traj.plot_pretty(times=times, ax=ax, color='green', label='best fit')
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)